In [1]:
import numpy as np
import pyscf
import matplotlib.pyplot as plt
import pyscf.lo


In [2]:
mol = pyscf.gto.M(atom="H 0. 0. 0.; H 0. 0. 1.4",
                  unit='B',
                  basis='minao')
mf = pyscf.scf.RKS(mol)
mf.kernel()

converged SCF energy = -1.09946580584267


-1.0994658058426698

In [19]:
def get_iao(mol, mf, mo_coeff, ref_bas = 'minao'):
  a = pyscf.lo.iao.iao(mol, mo_coeff, minao=ref_bas) # this is doing nothing but getting back the AO when we are using the minimal basis
  a = pyscf.lo.vec_lowdin(a, mf.get_ovlp()) # orthogonalize a
  R_ao_lo = np.einsum('ji,jk->ik', a, mf.get_ovlp())
  return a, R_ao_lo

a, R_ao_lo = get_iao(mol, mf, mf.mo_coeff[:,:2])
R_mo_lo = np.einsum('ji,jk,kl->il',a,mf.get_ovlp(),mf.mo_coeff[:,:2])

The TB Hamiltonian represents the Fock operator (kinetic + elec-ion + Hartree + exchange (or exchange-corrrelation in RKS)), while the int_1e represents only the kinetic + elec-ion. 

You may check that the int_1e in the AO basis does not depend on what method you use. 

In [5]:
# Obtain the 1e TB model Hamiltonian in LO
ham_tb_lo = np.einsum('i,ai,bi->ab', mf.mo_energy, R_mo_lo, R_mo_lo)
print('TB Ham:\n', ham_tb_lo)

# check that the TB model gives the same mo energies
eigvals, eigvecs = np.linalg.eigh(ham_tb_lo)
print('TB eigenvals:', eigvals)
print('HF mo energies:', mf.mo_energy)

TB Ham:
 [[-0.13344729 -0.29499569]
 [-0.29499569 -0.13344729]]
TB eigenvals: [-0.42844298  0.1615484 ]
HF mo energies: [-0.42844298  0.1615484 ]


In [6]:
# compute the 1e integrals
ham_1e_kin = pyscf.gto.getints("int1e_kin_sph", mol._atm, mol._bas, mol._env)
ham_1e_nuc = pyscf.gto.getints("int1e_nuc_sph", mol._atm, mol._bas, mol._env)

ham_1e = ham_1e_kin + ham_1e_nuc

# transform into MO basis
ham_1e_mo = np.einsum('kl,ki,lj', ham_1e, mf.mo_coeff, mf.mo_coeff)
print('direct 1e integrals in MO:\n', ham_1e_mo)


direct 1e integrals in MO:
 [[-1.18552106e+00 -3.33066907e-16]
 [-2.22044605e-16 -5.73440850e-01]]


In [23]:
print('direct 1e integrals in AO basis:\n', ham_1e)

direct 1e integrals in AO basis:
 [[-1.10989625 -0.9681947 ]
 [-0.9681947  -1.10989625]]


In [21]:
# transform into the localized basis
ham_1e_lo = np.einsum('kl,ki,lj', ham_1e, a, a)
print('direct 1e integrals in LO (orthogonalized AO):\n', ham_1e_lo)

direct 1e integrals in LO (orthogonalized AO):
 [[-0.87948095 -0.3060401 ]
 [-0.3060401  -0.87948095]]


In [9]:
# compute the 2e integrals
int_2e_ao = pyscf.gto.getints("int2e_sph", mol._atm, mol._bas, mol._env)
int_2e_mo = np.einsum('klmn,ka,lb,mc,nd->abcd', int_2e_ao, mf.mo_coeff, mf.mo_coeff, mf.mo_coeff, mf.mo_coeff)

print('direct 2e integrals in MO:\n',int_2e_mo)

direct 2e integrals in MO:
 [[[[ 5.66190189e-01 -8.32667268e-17]
   [-1.11022302e-16  5.56277523e-01]]

  [[ 5.55111512e-17  1.40192148e-01]
   [ 1.40192148e-01  3.33066907e-16]]]


 [[[-1.38777878e-16  1.40192148e-01]
   [ 1.40192148e-01  0.00000000e+00]]

  [[ 5.56277523e-01  0.00000000e+00]
   [ 0.00000000e+00  5.85863852e-01]]]]


In [10]:
# The density-density interactions betweem MO 0 and 0, 0 and 1 are close in value
print(int_2e_mo[0,0,0,0], int_2e_mo[1,1,1,1])

0.5661901887808966 0.5858638518021564


In [11]:
# The exchange interactions
print(int_2e_mo[0,1,1,0], int_2e_mo[1,0,0,1])

0.1401921481125708 0.14019214811256994


In [12]:
# transform to the localized orbital basis
int_2e_lo = np.einsum('klmn,ka,lb,mc,nd->abcd', int_2e_ao, a, a, a, a)

print('direct 2e integrals in LO:\n',int_2e_lo)

direct 2e integrals in LO:
 [[[[ 0.70634442 -0.00491842]
   [-0.00491842  0.42596012]]

  [[-0.00491842  0.00987475]
   [ 0.00987475 -0.00491842]]]


 [[[-0.00491842  0.00987475]
   [ 0.00987475 -0.00491842]]

  [[ 0.42596012 -0.00491842]
   [-0.00491842  0.70634442]]]]


##### Next, we compare the 1b and 2b energy contributions from the ab-initio (RKS) with those obtained using exact diagonalization of the model Hamiltonian

* RKS: 1b and 2b contributions 

In [25]:
rdm1_lo = np.einsum('ij,ai,bj->ab',mf.make_rdm1(), R_ao_lo, R_ao_lo)
e_1b = np.einsum('ij,ij->',ham_1e_lo, rdm1_lo)

rdm2_mo = np.zeros([2,2,2,2])
rdm2_mo[0,0,0,0] = 2
rdm2_lo = np.einsum('ijkl,ai,bj,ck,dl->abcd',rdm2_mo, R_mo_lo, R_mo_lo, R_mo_lo, R_mo_lo)
e_2b = 0.5*np.einsum('ijkl,ijkl->',int_2e_lo, rdm2_lo)

print('RKS (LDA) energy contribution from 1b and 2b:', e_1b, e_2b)
print('RKS total energy - E_nuc:', e_1b+e_2b)

RKS (LDA) energy contribution from 1b and 2b: -2.3710421147641005 0.5661901887808973
RKS total energy - E_nuc: -1.8048519259832032


* Exact diagonalization:

In [14]:
from pyscf import fci

def solver(n,h1,h2):
  nelec=(int(n/2),int(n/2))
  e, fcivec = fci.direct_spin1.kernel(h1, h2, n, nelec, verbose=5)
  dm1, dm2 = fci.direct_spin1.make_rdm12(fcivec, n, nelec)
  return {'e':e, 'dm1':dm1, 'dm2':dm2}

def get_energy_components(n,h1,h2):
  data = solver(n,h1,h2)
  int_expectation = 0.5*np.einsum('ijkl,ijkl->', data['dm2'], h2)
  oneb_expectation = np.einsum('ij,ij->', data['dm1'], h1)

  return {'total_energy_model':data['e'],  
          '1b_expectation':oneb_expectation,
          '2b_expectation':int_expectation} 

In [15]:
# Note that this total energy does not have the contribution from the nuclei interaction
get_energy_components(2,ham_1e_lo,int_2e_lo)

{'total_energy_model': -1.820457154669604,
 '1b_expectation': -2.3560596551758977,
 '2b_expectation': 0.5356025005062921}

In [26]:
# dmd model from optimized vmc
h1 = np.zeros([2,2])
h1[0,1] = -0.2894
h1[1,0] = h1[0,1]
h2 = np.zeros([2,2,2,2])
h2[0,0,0,0] = 0.1985
h2[1,1,1,1] = 0.1985
e0 = -0.6661

d = get_energy_components(2,h1,h2)
print('The ground state energy from ED of the DMD model:', d['total_energy_model'] + e0)

# This is to be compared with the total ground state energy
# Note that it is doing much better than the RKS

The ground state energy from ED of the DMD model: -1.154097820345039
